In [328]:
date(2004,10,1)

datetime.date(2004, 10, 1)

In [ ]:

import pandas as pd
import numpy as np

df_kl=pd.read_excel('~/Downloads/kl-1.xlsx')
df_kl.insert(loc=0,column='序号',value = [1,5,6,2,4,3])
df_kl_t=df_kl.sort_values(by=['序号'],ascending=True)
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','同期周末/平日比']]
page_2.astype({'周末/平日比':'float','同期周末/平日比':'float'})
page_2['周末占比同比'] = df_kl_t['周末/平日比'] - df_kl_t['同期周末/平日比']
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','周末占比同比']]

df_xs=pd.read_excel('~/Downloads/xs-1.xlsx')
df_all = df_xs.loc[(df_xs.项目分类 =='项目整体')]
df_all.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_a=df_all.sort_values(by=['序号'],ascending=True)
page_3 = df_xs_a[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_major = df_xs.loc[(df_xs.项目分类 =='主次力店')]
df_major.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_m=df_major.sort_values(by=['序号'],ascending=True)
page_4 = df_xs_m[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_normal = df_xs.loc[(df_xs.项目分类 =='一般店铺')]
df_normal.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_n=df_normal.sort_values(by=['序号'],ascending=True)
page_5 = df_xs_n[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]



page_1 = pd.merge(page_2,page_3,on='项目',how='left')[['项目','日均客流','日均客流同比','日均销售额','日均销额售同比']]


rjkl=page_2.日均客流.sum()/6
rjkl_tq = page_2.同期日均客流.sum()
rjkl_tb = (rjkl-rjkl_tq)/rjkl_tq

rjxs=page_3.日均销售额.sum()/6
rjxs_tq=page_3.同期日均销售额.sum()
rjxs_tb=(rjxs-rjxs_tq)/rjxs_tq

df_a_row1 = pd.DataFrame(np.insert(page_1.values,0,values=['均值',rjkl_tq,rjkl_tb,rjxs_tq,rjxs_tb],axis=0))

page_2=page_2.round(decimals=1)
page_3=page_3.round(decimals=0)
page_4=page_4.round(decimals=0)
page_5=page_5.round(decimals=0)
page_1=page_1.round(decimals=0)

with pd.ExcelWriter('~/Downloads/ppt.xlsx') as writer:
    df_a_row1.to_excel(writer,sheet_name='page_1')
    page_2.to_excel(writer,sheet_name='page_2')
    page_3.to_excel(writer,sheet_name='page_3')
    page_4.to_excel(writer,sheet_name='page_4')
    page_5.to_excel(writer,sheet_name='page_5')
    


### 半年报销售数据

In [329]:
from datetime import date, timedelta


def days_between(d1,d2):
    days_between = (d2-d1).days
    weekends, leftover = divmod(days_between,7)
    if leftover:
        start_day = (d2-timedelta(leftover)).isoweekday()
        end_day = start_day+leftover
        if start_day<=6 and end_day>6:
            weekends +=.5
        if start_day<=7 and end_day>7:
            weekends +=.5

    days_a = (d2 - d1).days
    days_w = weekends
    days_n =days_a - days_w*2
    
    return (days_a,days_n,days_w*2)




def get_diff_xs(df_xs_all_ppk1,df_xs_all_ppk2,form,mall,dt11,dt12,dt21,dt22):
    """
    Parameters: 
        1、dataframe: df_xs_all_ppk1 历史数据,df_xs_all_ppk2  当前数据 ；格式= ['项目名称','铺位类型','品牌名称','营业额','营业额日期','业态']
        
            get_diff_xs(df_xs_dj_202105,df_xs_dj_202205,'合肥弘阳广场',0)
        2、form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    # 
    dw = days_between(date(2022,1,1),date(2022,7,1))
    print(dw)
    c_zm1 = dw[2]
    c_pr1 = dw[1]
    
    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    print((c_zm1,c_pr1))
    
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/c_zm1 if x['周末']=='是' else x['营业额']/c_pr1,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':[mall],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    c_zm2 = dw[2]
    c_pr2 = dw[1]

    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/c_zm2 if x['周末']=='是' else x['营业额']/c_pr2,axis=1)

    print(df_tmp2)
    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]

    df_xs_res2 = pd.DataFrame(data={'项目名称':[mall],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

In [330]:

import pandas as pd
import numpy as np

df_xs_yt=pd.read_excel('~/Desktop/销售客流pandas/业态202201_06.xlsx',header=1)
df_xs_yt = df_xs_yt.loc[(df_xs_yt.项目 == '南京弘阳广场') | (df_xs_yt.项目 == '合肥弘阳广场') | (df_xs_yt.项目 == '常州中吴弘阳广场') | (df_xs_yt.项目 == '济南弘阳广场') | (df_xs_yt.项目 == '燕郊弘阳广场') | (df_xs_yt.项目 == '衡阳弘阳广场')]

# df_xs_yt = df_xs_yt[['项目整体','项目','一月','二月','三月','四月','五月','六月']]
# # df_xs_yt
# yetai = ['主次力店','餐饮','儿童','服装','精品','娱乐','服务']

# df_res = []
# for yt in yetai:
#     df = df_xs_yt.loc[(df_xs_yt.项目整体 == yt)]
#     df.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
#     df=df.sort_values(by=['序号'],ascending=True)
#     d = df[['项目整体','项目','一月','二月','三月','四月','五月','六月']]
#     x = d.apply(pd.to_numeric,errors='ignore')
#     df_res.append(x)

# with pd.ExcelWriter('~/Desktop/销售客流pandas/ppt7.xlsx') as writer:
#     i = 0
#     for yt in yetai:
#         df_res[i].round(decimals=0).to_excel(writer,sheet_name= yt,index=False)
#         i += 1

# 南京弘阳广场2021 和 2022 销售数据
df1 = pd.read_excel('~/Desktop/销售客流pandas/正铺202101_06.xls')
df2 = pd.read_excel('~/Desktop/销售客流pandas/多经202101_06.xls')
df3 = pd.read_excel('~/Desktop/销售客流pandas/正铺202201_06.xls')
df4 = pd.read_excel('~/Desktop/销售客流pandas/多经202201_06.xls')

df2021 = pd.concat([df1,df2])

df2022 = pd.concat([df3,df4])


df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
df2021 =df2021[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
df_ppk = df_ppk[['品牌名称','业态']]
df2021_ppk = pd.merge(df2021,df_ppk,on='品牌名称',how='left')
df2021_ppk['月份'] = df2021_ppk['营业额日期'].dt.month
df2021_ppk_res = df2021_ppk.groupby(by=['项目名称','月份']).agg({'营业额':'sum'}).reset_index()
print(df2021_ppk_res.T)

df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
df2022 =df2022[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
df_ppk = df_ppk[['品牌名称','业态']]
df2022_ppk = pd.merge(df2022,df_ppk,on='品牌名称',how='left')

df2022_ppk['月份'] = df2022_ppk['营业额日期'].dt.month
df2022_ppk_res = df2022_ppk.groupby(by=['项目名称','月份']).agg({'营业额':'sum'}).reset_index()



# with pd.ExcelWriter('~/Desktop/销售客流pandas/ppt4.xlsx') as writer:
#     i = 0
#     for yt in yetai:
#         dtmp = df2022_ppk_res.loc[(df2022_ppk_res['业态'].str.strip() == yt)]
#         dtmp['营业额'] = dtmp['营业额']/10000
#         dtmp.round(0)
#         dtmp.T.to_excel(writer,sheet_name= yt,index=False)
#         i += 1

d = days_between(date(2022,1,1),date(2022,7,1))

# 2022 1-6月 日均销售
def get_xs_rijun(df2022_pk):
    df2022_ppk = df2022_pk.groupby(by=['项目名称','营业额日期']).agg({'营业额':'sum'}).reset_index()
    df2022_ppk['营业额'] = df2022_pk['营业额']/10000
    dff = df2022_pk.groupby(by=['项目名称']).agg({'营业额':'sum'}).reset_index()
    dff['总销售额']  = dff['营业额']
    dff['日均销售额']=dff['营业额']/d[0]
    dff['天数'] = d[0]
    return dff

# 2022 1-6月 各业态销售
def get_xs_by_yt(df2022_pk):
    
    ## 1、按项目、业态、月份分组汇总
    df2022_pk['月份'] = df2022_pk['营业额日期'].dt.month
    df2022_pk_yt = df2022_pk.groupby(by=['项目名称','业态','月份']).agg({'营业额':'sum'}).reset_index()
    df_nj = df2022_pk_yt.sort_values(by=['项目名称','业态','月份'])
    from functools import reduce

    ## 2、行转列、合并、整理
    yetai = ['主次力店','餐饮','儿童','服装','精品','娱乐','服务']
    df_r = pd.DataFrame()
    dflist = []
    for yt in yetai :
        dtmp = df_nj.loc[(df_nj.业态.str.strip() == yt)][['月份','营业额']].T.reset_index()
        dtmp.columns = ['项目','一月','二月','三月','四月','五月','六月']
        dr = dtmp[1:].copy()
        dr['项目'] = '南京弘阳广场'
        dr[['项目整体']] = yt
        dflist.append(dr)
    
    df_res = reduce(lambda left,right:pd.concat([left,right],ignore_index=True),dflist)
    df_res['一月'] = df_res['一月']/10000
    df_res['二月'] = df_res['二月']/10000
    df_res['三月'] = df_res['三月']/10000
    df_res['四月'] = df_res['四月']/10000
    df_res['五月'] = df_res['五月']/10000
    df_res['六月'] = df_res['六月']/10000
    df_res['总销售额'] = df_res['一月'] +df_res['二月'] +df_res['三月'] +df_res['四月']+df_res['五月']+df_res['六月']
    return  df_res.round(0)

# df_yt_2021 = get_xs_by_yt(df2021_ppk)
df_yt_2022 = get_xs_by_yt(df2022_ppk)
df_rj_2021 = get_xs_rijun(df2021_ppk)
df_rj_2022 = get_xs_rijun(df2022_ppk)
dt11,dt12,dt21,dt22=date(2021,1,1),date(2021,7,1),date(2022,1,1),date(2022,7,1)
df_all = get_diff_xs(df2021_ppk,df2022_ppk,0,'南京弘阳广场',dt11,dt12,dt21,dt22)
# df_zcl = get_diff_xs(df2021_ppk,df2022_ppk,1,'南京弘阳广场')
# df_yb = get_diff_xs(df2021_ppk,df2022_ppk,2,'南京弘阳广场')
# df_zcl
df_rj_2021.round(0)


                0               1            2            3            4  \
项目名称       南京弘阳广场          南京弘阳广场       南京弘阳广场       南京弘阳广场       南京弘阳广场   
月份              1               2            3            4            5   
营业额   79348000.01  121862143.5773  84168321.23  81627458.69  88936533.05   

                5  
项目名称       南京弘阳广场  
月份              6  
营业额   78117192.73  
(181, 129.0, 52.0)
(52.0, 129.0)
  周末           营业额         日均营业额
0  否  3.194167e+08  2.476098e+06
1  是  2.237631e+08  4.303137e+06


,项目名称,营业额,总销售额,日均销售额,天数
0,南京弘阳广场,534059649.0,534059649.0,2950606.0,181


In [331]:
df_rj_2022

,项目名称,营业额,总销售额,日均销售额,天数
0,南京弘阳广场,5.431798e+08,5.431798e+08,3.000993e+06,181


In [332]:
df_all


,index,项目名称,平日销售日均,周末销售日均,周末/平日比,上期周末占比,周末占比同比
0,0,南京弘阳广场,2476098.48,4303136.87,0.7,0.72,-0.02


In [282]:
df_yt_2022

In [283]:
df2022_ppk

,项目名称,铺位类型,品牌名称,营业额,营业额日期,业态,月份,周末
0,南京弘阳广场,正铺,WLC glasses,1316.0,2022-06-30,精品,6,否
1,南京弘阳广场,正铺,高丽苑韩国烤肉,14572.0,2022-06-30,餐饮,6,否
2,南京弘阳广场,正铺,WIN BEAUTY,8800.0,2022-06-30,精品,6,否
3,南京弘阳广场,正铺,魔法师,3466.0,2022-06-30,餐饮,6,否
4,南京弘阳广场,正铺,安踏kids,2816.0,2022-06-30,儿童,6,否
...,...,...,...,...,...,...,...,...
80011,南京弘阳广场,多经,海底捞,0.0,2022-01-01,主次力店,1,是
80012,南京弘阳广场,多经,小菜园,0.0,2022-01-01,餐饮,1,是
80013,南京弘阳广场,多经,海底捞,0.0,2022-01-01,主次力店,1,是
80014,南京弘阳广场,多经,韩京海鲜烤肉自助餐厅,0.0,2022-01-01,餐饮,1,是


### 客流：平日/周末客流统计

In [13]:
import pandas as pd
import numpy as np

df_1 = pd.read_excel('~/Desktop/销售客流pandas/南京弘阳广场客流202101-06.xls',header=5)
df_a1= df_1.loc[(df_1.属性 =='平时') | (df_1.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})

df_ps1= df_a1.loc[(df_a1.属性 =='平时')][['日期','星期','属性','客流']].sort_values(by='日期')
df_zm1= df_a1.loc[(df_a1.属性 =='周末')][['日期','星期','属性','客流']].sort_values(by='日期')
df_a1['日期'] = pd.to_datetime(df_a1['日期'],format='%Y-%m-%d')
df_a1['月份'] =df_a1['日期'].dt.month
r1 = df_a1.groupby(by=['月份']).agg({'客流':'sum'}).reset_index()

df_ps1=df_ps1.groupby(df_ps1.属性).agg({'客流':'sum'}).reset_index()
df_zm1=df_zm1.groupby(df_zm1.属性).agg({'客流':'sum'}).reset_index()

df_2 = pd.read_excel('~/Desktop/销售客流pandas/南京弘阳广场客流202201-06.xls',header=5)
df_a2= df_2.loc[(df_2.属性 =='平时') | (df_2.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'}).reset_index()
df_ps2= df_a2.loc[(df_a2.属性 =='平时')][['日期','星期','属性','客流']].sort_values(by='日期')
df_zm2= df_a2.loc[(df_a2.属性 =='周末')][['日期','星期','属性','客流']].sort_values(by='日期')
df_ps2=df_ps2.groupby(df_ps2.属性).agg({'客流':'sum'}).reset_index()
df_zm2=df_zm2.groupby(df_zm2.属性).agg({'客流':'sum'}).reset_index()

df_a2['日期'] = pd.to_datetime(df_a2['日期'],format='%Y-%m-%d')
df_a2['月份'] =df_a2['日期'].dt.month
r2 = df_a2.groupby(by=['月份']).agg({'客流':'sum'}).reset_index()
# r2.T
(df_ps1,df_zm1)

(   属性        客流
 0  平时  12358125,
    属性       客流
 0  周末  9335018)

### 根据品牌库、销售导出数据 计算

In [6]:
import pandas as pd
import numpy as np
from output_style_format import to_excel_auto_column_weight

def get_df(folderpath,shoppingmall,month:str):
    #folderpath,month1,month2,shoppingmall,form
    df_xs_all = pd.read_excel(folderpath+shoppingmall+month+'.xlsx')
    df_ppk = pd.read_excel(folderpath+'/品牌库.xlsx')


    df_xs_all =df_xs_all[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_ppk = df_ppk[['品牌名称','业态']]

    df_xs_all_ppk = pd.merge(df_xs_all,df_ppk,on='品牌名称',how='left')

    df_xs_zcld = df_xs_all_ppk.loc[(df_xs_all_ppk.业态=='主次力店')]
    df_xs_zcld = df_xs_zcld.groupby(df_xs_zcld.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_ybdp = df_xs_all_ppk.loc[(df_xs_all_ppk.业态!='主次力店')]
    df_xs_ybdp = df_xs_ybdp.groupby(df_xs_ybdp.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_zcld['营业额日期']=df_xs_zcld['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))
    df_xs_ybdp['营业额日期']=df_xs_ybdp['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))

    return df_xs_zcld,df_xs_ybdp

df1,df2=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202105')
df5,df6=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202205')

with pd.ExcelWriter('~/Desktop/销售客流pandas/xs_合肥弘阳广场1.xlsx',engine="openpyxl") as writer:
    to_excel_auto_column_weight(df1,writer,'主次力店202105')
    to_excel_auto_column_weight(df2,writer,'一般店铺202105')
    to_excel_auto_column_weight(df5,writer,'主次力店202205')
    to_excel_auto_column_weight(df6,writer,'一般店铺202205')



In [36]:
import pandas as pd
import numpy as np

from datetime import datetime


def get_diff_xs(folderpath,month1,month2,shoppingmall,form):
    """
    Parameters: Str: month1,month2,shoppingmall
    get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
    form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #导入数据并整理
    df_xs_zp = pd.read_excel(folderpath+shoppingmall+month1+'.xlsx')
    df_xs_dj = pd.read_excel(folderpath+shoppingmall+month2+'.xlsx')
    df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
    df_xs_zp =df_xs_zp[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_dj =df_xs_dj[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_zp['营业额日期']=df_xs_zp.营业额日期.apply(lambda x:x.date())
    df_xs_dj['营业额日期']=df_xs_dj.营业额日期.apply(lambda x:x.date())

    df_ppk = df_ppk[['品牌名称','业态']]
    df_xs_all_ppk1 = pd.merge(df_xs_zp,df_ppk,on='品牌名称',how='left')
    df_xs_all_ppk2 = pd.merge(df_xs_dj,df_ppk,on='品牌名称',how='left')
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]
    df_xs_res2 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

df_all = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
df_zcld = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',1)
df_ybsp = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',2)

with pd.ExcelWriter('~/Downloads/xs_hefei.xlsx') as writer:
    df_all.to_excel(writer,sheet_name='所有业态')
    df_zcld.to_excel(writer,sheet_name='主次力店')
    df_ybsp.to_excel(writer,sheet_name='一般商铺')


In [ ]:
import pandas as pd
import numpy as np

df_tj =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='统计表',header=2)
df_hf_202105 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202105')
df_hf_202205 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202205')

df1=df_tj.iloc[:,1:]
df2=df_tj.iloc[:,:4]

with pd.ExcelWriter('~/Downloads/11111.xlsx') as writer:
    df1.to_excel(writer,sheet_name='s1')
    df2.to_excel(writer,sheet_name='s1',startrow=4)

In [21]:

dd = df_jy[['客户编号']]
dd

,客户编号
0,IFCA
1,AT-190317-12783-3
2,AT-190317-12898-36
3,AT-190317-13177-16
4,AT-190317-13453-12
...,...
12331,KH20226291819
12332,TE010696
12333,TE011587
12334,KH2022771841


In [10]:
import pandas as pd
import numpy as np

df_jy = pd.read_excel('~/Downloads/客户/客户_竞优20220715.xls')
dd_jy = df_jy[['客户编码','客户姓名','法人身份证号码']]
df_zz = pd.read_excel('~/Downloads/客户/客户_总账20220715.xlsx')
df_jd = pd.read_excel('~/Downloads/客户/客户_金蝶20220715.xlsx')

# dd_jd = df_jd[['客户编号','名称']]
df = pd.merge(df_jd,df_zz,on='客户编码',how='left') # 先筛选金蝶中没被总账引用的客户，然后拿筛选结果去对比竞优的数据，对比结果中竞优没有的就要 反核准。

df1 =  df.loc[(pd.isna(df.创建人))] # df1 金蝶中未被总账使用的客户

df2 = pd.merge(df1,df_jy,on='客户编码',how='left')
df3  = df2.loc[(pd.isna(df2.客户姓名))]
with pd.ExcelWriter('~/Downloads/客户/竞优无-07151616.xlsx') as writer:
    df3.to_excel(writer,sheet_name='1')


In [2]:
import pandas as pd
import numpy as np

df_jczl = pd.read_excel('~/Downloads/客户/jczl.xlsx')
df_jczl = df_jczl[['编码','名称','税务登记号','创建时间']]
df_jczl['客户编码'] = df_jczl['编码']

df_cwzl = pd.read_excel('~/Downloads/客户/cwzl.xlsx')
df_cwzl = df_cwzl[['客户编码','财务组织编码']]
df = pd.merge(df_jczl,df_cwzl,on='客户编码',how='right')

df =  df.loc[(pd.isna(df.名称))]

with pd.ExcelWriter('~/Downloads/客户/竞优无.xlsx') as writer:
    df.to_excel(writer,sheet_name='1')

In [ ]:
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts


table = Table()

headers = ["City name", "Area", "Population", "Annual Rainfall"]
rows = [
    ["Brisbane", 5905, 1857594, 1146.4],
    ["Adelaide", 1295, 1158259, 600.5],
    ["Darwin", 112, 120900, 1714.7],
    ["Hobart", 1357, 205556, 619.5],
    ["Sydney", 2058, 4336374, 1214.8],
    ["Melbourne", 1566, 3806092, 646.9],
    ["Perth", 5386, 1554769, 869.4],
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title="Table-基本示例", subtitle="我是副标题支持换行哦")
)
table.render_notebook()

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Sankey

nodes = [
    {"name": "category1"},
    {"name": "category2"},
    {"name": "category3"},
    {"name": "category4"},
    {"name": "category5"},
    {"name": "category6"},
]

links = [
    {"source": "category1", "target": "category2", "value": 10},
    {"source": "category2", "target": "category3", "value": 15},
    {"source": "category3", "target": "category4", "value": 20},
    {"source": "category5", "target": "category6", "value": 25},
]
c = (
    Sankey()
    .add(
        "sankey",
        nodes,
        links,
        linestyle_opt=opts.LineStyleOpts(opacity=0.2, curve=0.5, color="source"),
        label_opts=opts.LabelOpts(position="right"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Sankey-基本示例"))
    .render_notebook()
)

c

In [38]:
df = pd.read_clipboard()
df


,name,age,city,sex,income
0,Tom,30,北京,male,3000
1,Bob,30,上海,male,8000
2,Mary,35,广州,female,8000
3,James,19,克利夫兰,male,4000
4,Andy,22,郑州,female,6000
5,Alice,30,晋城,female,7000
6,Kobe,37,洛杉矶,male,10000
7,Yafei,25,晋城,male,70000


In [27]:

(
    df.loc[:, ['content']] # 选取标识列，与下行时间转换后形成新的 df
    .assign(time=pd.to_datetime(df['date'] + ' ' +df['time']))
    # 时间分组，每两分钟分组到一起
    .groupby(pd.Grouper(freq='2Min', key='time'))
    # 聚合方法为计算标识的数量
    .apply(lambda x: x.content.value_counts())
    .unstack() # 解除堆叠
    .fillna(0)
    .astype(int).reset_index()
)

,time,A,B,C,D,E,F
0,2021-08-12 22:22:00+08:00,1,1,0,0,0,0
1,2021-08-12 22:34:00+08:00,1,0,0,1,0,0
2,2021-08-12 22:36:00+08:00,0,0,1,0,1,0
3,2021-08-12 22:38:00+08:00,0,0,0,0,1,1
4,2021-08-12 22:40:00+08:00,0,0,1,0,1,0
5,2021-08-12 22:42:00+08:00,0,2,0,0,0,0
6,2021-08-12 22:44:00+08:00,1,0,0,0,0,0


In [28]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
                                   ('two', 'a'), ('two', 'b')])
s = pd.Series(np.arange(1.0, 5.0), index=index)

s

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [30]:
f=s.T
f

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [33]:
s.unstack(0)

,one,two
a,1.0,3.0
b,2.0,4.0


In [60]:
pd.date_range("2022-07-06", periods=10, freq="B")

DatetimeIndex(['2022-07-06', '2022-07-08', '2022-07-12', '2022-07-14',
               '2022-07-18', '2022-07-20', '2022-07-22', '2022-07-26',
               '2022-07-28', '2022-08-01'],
              dtype='datetime64[ns]', freq='2B')

In [61]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum)

df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [2]:
import pandas as pd

df = pd.read_clipboard()
df

,name,age,city,sex,income
0,Tom,30,北京,male,3000
1,Bob,30,上海,male,8000
2,Mary,35,广州,female,8000
3,James,19,克利夫兰,male,4000
4,Andy,22,郑州,female,6000
5,Alice,30,晋城,female,7000
6,Kobe,37,洛杉矶,male,10000
7,Yafei,25,晋城,male,70000


In [6]:
user_info_reset_index = df
user_info_reset_index.melt(id_vars=['name'], value_vars=['sex'])

,name,variable,value
0,Tom,sex,male
1,Bob,sex,male
2,Mary,sex,female
3,James,sex,male
4,Andy,sex,female
5,Alice,sex,female
6,Kobe,sex,male
7,Yafei,sex,male


In [ ]:
import pandas as pd
import numpy as np

df_wu = pd.read_excel('~/Downloads/客户/金蝶有竞优无-金蝶总账未引用.xlsx')

df_jd = pd.read_excel('~/Downloads/客户/客户20220715172039.xlsx')

# dd_jd = df_jd[['客户编号','名称']]
df = pd.merge(df_wu,df_jd,on='客户编码',how='left') # 先筛选金蝶中没被总账引用的客户，然后拿筛选结果去对比竞优的数据，对比结果中竞优没有的就要 反核准。

df1 =  df.loc[(pd.isna(df.名称2))] # df1 金蝶中未被总账使用的客户

df1


In [7]:
import pandas as pd
import tushare as ts
import numpy as np
pro =  ts.pro_api('5e4e8f2545d9be88ec337aa1f798038a7d0cf9751eafc5afa13e6a08')
df = pro.daily(ts_code='300274.sz', start_date='20180701', end_date='20180718')
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,300274.SZ,20180718,8.09,8.15,7.79,7.80,8.06,-0.26,-3.23,511840.32,406140.634
1,300274.SZ,20180717,8.11,8.13,7.91,8.06,8.06,0.00,0.00,245180.80,196423.949
2,300274.SZ,20180716,8.19,8.30,8.01,8.06,8.13,-0.07,-0.86,347926.26,282298.863
3,300274.SZ,20180713,8.30,8.35,8.12,8.13,8.28,-0.15,-1.81,277269.61,227513.160
4,300274.SZ,20180712,7.96,8.31,7.96,8.28,7.93,0.35,4.41,366705.36,299943.494
5,300274.SZ,20180711,8.06,8.17,7.83,7.93,8.23,-0.30,-3.65,344808.21,275285.051
6,300274.SZ,20180710,8.62,8.63,8.03,8.23,8.57,-0.34,-3.97,553768.57,456974.841
7,300274.SZ,20180709,8.50,8.57,8.43,8.57,8.42,0.15,1.78,221854.25,188982.194
8,300274.SZ,20180706,8.34,8.56,8.08,8.42,8.33,0.09,1.08,387707.69,323121.304
9,300274.SZ,20180705,8.73,8.82,8.33,8.33,8.68,-0.35,-4.03,293114.78,250033.559


In [6]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("300274.sz",
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
    start_date='2017-07-01', end_date='2018-7-18',
    frequency="d", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)
#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

#### 结果集输出到csv文件 ####   
result.to_csv("~/Downloads/history_A_stock_k_data.csv", index=False)
print(result)

#### 登出系统 ####
bs.logout()


login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
           date       code     open     high      low    close preclose  \
0    2017-07-03  sz.300274  11.1400  11.3800  11.0600  11.3500  11.0900   
1    2017-07-04  sz.300274  11.3400  11.3400  11.1900  11.2400  11.3500   
2    2017-07-05  sz.300274  11.2400  11.3000  11.1100  11.2300  11.2400   
3    2017-07-06  sz.300274  11.2200  11.2900  10.9800  11.1000  11.2300   
4    2017-07-07  sz.300274  11.0800  11.1300  10.9100  11.1000  11.1000   
..          ...        ...      ...      ...      ...      ...      ...   
252  2018-07-12  sz.300274   7.9600   8.3100   7.9600   8.2800   7.9300   
253  2018-07-13  sz.300274   8.3000   8.3500   8.1200   8.1300   8.2800   
254  2018-07-16  sz.300274   8.1900   8.3000   8.0100   8.0600   8.1300   
255  2018-07-17  sz.300274   8.1100   8.1300   7.9100   8.0600   8.0600   


In [8]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("sz.300274",
    "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date='2022-07-01', end_date='2022-07-16',
    frequency="5", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

#### 结果集输出到csv文件 ####   
result.to_excel("~/Downloads/history_A_stock_k_data.xlsx", index=False)
print(result)

#### 登出系统 ####
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
           date               time       code      open      high       low  \
0    2022-07-01  20220701093500000  sz.300274   98.5000   99.5000   97.1000   
1    2022-07-01  20220701094000000  sz.300274   99.5700   99.6000   97.7200   
2    2022-07-01  20220701094500000  sz.300274   98.4800   98.8900   97.5200   
3    2022-07-01  20220701095000000  sz.300274   97.9600   98.3700   97.8500   
4    2022-07-01  20220701095500000  sz.300274   97.9000   97.9000   96.7800   
..          ...                ...        ...       ...       ...       ...   
523  2022-07-15  20220715144000000  sz.300274  127.5700  127.5700  126.3100   
524  2022-07-15  20220715144500000  sz.300274  127.1300  127.1300  125.7700   
525  2022-07-15  20220715145000000  sz.300274  125.7200  126.5400  125.6600   
526  2022-07-15  20220715145500000 

In [9]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取行业分类数据
rs = bs.query_stock_industry()
# rs = bs.query_stock_basic(code_name="浦发银行")
print('query_stock_industry error_code:'+rs.error_code)
print('query_stock_industry respond  error_msg:'+rs.error_msg)

# 打印结果集
industry_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    industry_list.append(rs.get_row_data())
result = pd.DataFrame(industry_list, columns=rs.fields)
# 结果集输出到csv文件
result.to_excel("~/Downloads/hyfl.xlsx", encoding="gbk", index=False)
print(result)

# 登出系统
bs.logout()


login success!
login respond error_code:0
login respond  error_msg:success
query_stock_industry error_code:0
query_stock_industry respond  error_msg:success
      updateDate       code code_name industry industryClassification
0     2022-06-13  sh.600000      浦发银行       银行                 申万一级行业
1     2022-06-13  sh.600001      邯郸钢铁                          申万一级行业
2     2022-06-13  sh.600002      齐鲁石化                          申万一级行业
3     2022-06-13  sh.600003     ST东北高                          申万一级行业
4     2022-06-13  sh.600004      白云机场     交通运输                 申万一级行业
...          ...        ...       ...      ...                    ...
4881  2022-06-13  sz.301266      宇邦新材                          申万一级行业
4882  2022-06-13  sz.301268       铭利达                          申万一级行业
4883  2022-06-13  sz.301279      金道科技     机械设备                 申万一级行业
4884  2022-06-13  sz.301288      清研环境                          申万一级行业
4885  2022-06-13  sz.301298      东利机械                          申万一级行业

[4

In [11]:
bs.login()
date = '2022-07-15'
# 获取指定日期的指数、股票数据
stock_rs = bs.query_all_stock('2022-07-15')
stock_df = stock_rs.get_data()
data_df = pd.DataFrame()
for code in stock_df["code"]:
    print("Downloading :" + code)
    k_rs = bs.query_history_k_data_plus(code, "date,code,open,high,low,close", date, date)
    data_df = data_df.append(k_rs.get_data())
bs.logout()
data_df.to_excel("~/Downloads/allstockbydate.xlsx", encoding="gbk", index=False)
print(data_df)


login success!
logout success!
          date       code        open        high         low       close
0   2022-07-15  sh.000001   3261.3826   3288.9198   3228.0609   3228.0609
0   2022-07-15  sh.000002   3417.6583   3446.5006   3382.6849   3382.6849
0   2022-07-15  sh.000003    303.5837    305.3727    300.0219    300.0659
0   2022-07-15  sh.000004   3091.6628   3130.7893   3070.7728   3070.7728
0   2022-07-15  sh.000005   2598.4682   2608.6378   2557.3069   2557.3069
..         ...        ...         ...         ...         ...         ...
0   2022-07-15  sz.399994   1368.9273   1375.7657   1347.1631   1347.1796
0   2022-07-15  sz.399995   4249.2024   4249.4416   4134.5201   4134.5201
0   2022-07-15  sz.399996   2617.8651   2667.1783   2610.8527   2617.6839
0   2022-07-15  sz.399997  16080.9915  16612.4648  16036.3697  16174.3932
0   2022-07-15  sz.399998   2344.4311   2423.1109   2323.9131   2323.9131

[5233 rows x 6 columns]


In [3]:
from jqdatasdk import *

auth('17712910405','Momocha_1982') #账号是申请时所填写的手机号；密码为聚宽官网登录密码

Exception: 未开通权限;
如果未开通调用权限，请打开以下链接提交申请：https://www.joinquant.com/default/index/sdk